In [25]:
import spacy
import pandas as pd

In [41]:
nlp = spacy.load('en_core_web_sm')

In [42]:
df = pd.read_csv('reddit_investing.csv', sep='|')

In [43]:
df.head()

,name,created_utc,subreddit,selftext,upvote_ratio,ups,downs,score
0,t3_119u28i,1.677146e+09,investing,Have a general question? Want to offer some c...,0.75,4.0,0.0,4.0
1,t3_119u7zq,1.677147e+09,investing,"Hi everyone, I just created my Roth IRA accoun...",0.89,132.0,0.0,132.0
2,t3_1191h6s,1.677077e+09,investing,https://www.barrons.com/articles/intel-cuts-di...,0.96,1325.0,0.0,1325.0
3,t3_119ibx7,1.677110e+09,investing,a) Results were in line with revised expectati...,0.89,184.0,0.0,184.0
4,t3_119yxek,1.677162e+09,investing,"I don’t know about you, but I’m sick and tired...",0.62,9.0,0.0,9.0


In [44]:
def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

In [45]:
df = df.iloc[:500]

In [46]:
df.shape

(500, 8)

In [57]:
df['organizations'] = df['selftext'].apply(get_orgs)

In [48]:
df

,name,created_utc,subreddit,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_119u28i,1.677146e+09,investing,Have a general question? Want to offer some c...,0.75,4.0,0.0,4.0,"[FAQ, wiki]"
1,t3_119u7zq,1.677147e+09,investing,"Hi everyone, I just created my Roth IRA accoun...",0.89,132.0,0.0,132.0,[]
2,t3_1191h6s,1.677077e+09,investing,https://www.barrons.com/articles/intel-cuts-di...,0.96,1325.0,0.0,1325.0,"[AMD, FF, Intel, gt;Intel]"
3,t3_119ibx7,1.677110e+09,investing,a) Results were in line with revised expectati...,0.89,184.0,0.0,184.0,"[AI, Nvidia, here.](https://www.reddit.com]"
4,t3_119yxek,1.677162e+09,investing,"I don’t know about you, but I’m sick and tired...",0.62,9.0,0.0,9.0,[]
...,...,...,...,...,...,...,...,...,...
495,t3_1118j6p,1.676296e+09,investing,"Hello,\n\nEvery month I put an equal amount in...",0.52,1.0,0.0,1.0,"[VanEck, IWDA, Vanguard, iShares, All World]"
496,t3_10zzpsp,1.676154e+09,investing,I went with just the first quarter because IMH...,0.89,783.0,0.0,783.0,[]
497,t3_1114xqh,1.676283e+09,investing,"Hi everyone, I have a quick question. \n\nIf s...",0.56,1.0,0.0,1.0,[]
498,t3_111c2a9,1.676303e+09,investing,You've probably all see then the line chart sh...,0.24,0.0,0.0,0.0,[FED]


In [49]:
### extract most mentioned entities 

from collections import Counter

In [58]:
orgs = df['organizations'].to_list()

In [59]:
#convert a list of list to a list

orgs = [org for sublist in orgs for org in sublist]

In [60]:
org_freq = Counter(orgs)

In [61]:
org_freq.most_common(10)

[('wiki', 29),
 ('Vanguard', 19),
 ('Google', 16),
 ('Fidelity', 15),
 ('AI', 14),
 ('VOO', 14),
 ('VMSXX', 9),
 ('Microsoft', 8),
 ('IRS', 8),
 ('BLS', 8)]

In [56]:

blacklist = ['FAQ', 'FED']

def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG' and entity.text.upper() not in blacklist:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

In [63]:
df.to_csv('reddit_investing_ner.csv', sep ='|', index=False)